In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import llama_cpp
import torch

from curverag import utils
from curverag.curverag import CurveRAG, DEFAULT_ENTITY_TYPES, DEFAULT_GLINER_MODEL, DEFAULT_SENTENCE_TRANSFORMER_MODEL
from curverag.graph import KnowledgeGraph
from curverag.atth.kg_dataset import KGDataset
from curverag.atth.models.hyperbolic import AttH
from curverag.eval import evaluation, queries, context, expected_output

# Explore dataset

In [3]:
with open('/Users/nathan/Documents/projects/datasets/2WikiMultihopQA/train.json', 'rb') as f:
    train = json.load(f)

with open('/Users/nathan/Documents/projects/datasets/2WikiMultihopQA/dev.json', 'rb') as f:
    dev = json.load(f)

with open('/Users/nathan/Documents/projects/datasets/2WikiMultihopQA/test.json', 'rb') as f:
    test = json.load(f)

In [4]:
train[0].keys()

dict_keys(['_id', 'type', 'question', 'context', 'supporting_facts', 'evidences', 'answer'])

In [5]:
train[0]['question']

'Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?'

In [6]:
train[0]['answer']

'no'

In [7]:
train[0]['context']

[['Stuart Rosenberg',
  ['Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984).',
   'He was noted for his work with actor Paul Newman.']],
 ['Méditerranée (1963 film)',
  ['Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff.',
   'It was written by Philippe Sollers and produced by Barbet Schroeder, with music by Antione Duhamel.',
   'The 45 minute film is cited as one of Pollet\'s most influential films, which according to Jonathan Rosenbaum directly influenced Jean-Luc Goddard\'s "Contempt", released later the same year.',
   'Footage for the film was shot around the Mediterranean, including at a Greek temple, a Sicilian garden, the sea, and also features a fisherman, a bullfighter, and a girl on an operating table

# Run eval

In [8]:
max_tokens = 10000
n_ctx=10000

llm, outlines_model = utils.load_model(
    llm_model_path="./models/Meta-Llama-3-8B-Instruct.Q6_K.gguf",
    tokenizer=llama_cpp.llama_tokenizer.LlamaHFTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct"),
    n_ctx=n_ctx,
    max_tokens=max_tokens
)

llama_model_load_from_file_impl: using device Metal (Apple M4 Pro) - 16383 MiB free
llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from ./models/Meta-Llama-3-8B-Instruct.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:            

In [9]:
entity_types = ['people', 'locations', 'entities', 'movies', 'directors']
rag = CurveRAG(
    llm,
    outlines_model,
    entity_types,
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/nathan/Documents/projects/curve_rag/.venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [11]:
new_docs = []
for d in train[0]['context']:
    d_ = str(d).replace('[', '').replace(']', '. ').replace('\'', '')
    new_docs.append(d_)

In [19]:
new_docs = []
for d in train:
    d_ = str(d['context']).replace('[', '').replace(']', '. ').replace('\'', '')
    new_docs.append(d_)

In [23]:
new_docs[1]

'The Falcon Takes Over, The Falcon Takes Over( also known as The Falcon Steps Out), is a 1942 black- and- white mystery film directed by Irving Reis., The B film was the third, following" The Gay Falcon" and" A Date with the Falcon"( 1941), to star George Sanders as the character Gay Lawrence, a gentleman detective known by the sobriquet the Falcon.. . , The Falcon Strikes Back, The Falcon Strikes Back( The Falcon Comes Back) is a 1943 American crime film directed by Edward Dmytryk and stars Tom Conway as the title character, the amateur sleuth, the Falcon., Supporting roles are filled by Harriet Hilliard, Jane Randolph, Edgar Kennedy, with Cliff Edwards filling in for Allen Jenkins as the Falcon\\s sidekick," Goldie" Locke., It is the sixth film in the Falcon series and the second for Conway, reprising the role that his brother, George Sanders had initiated.. . , Vatroslav Mimica, Vatroslav Mimica( born 25 June 1923) is a Croatian film director and screenwriter., Born in Omiš, Mimica 

In [13]:
'''docs = [
    "The patient was diagnosed with type 2 diabetes mellitus and prescribed metformin 500mg twice daily.",
    "MRI scan revealed a small lesion in the left temporal lobe suggestive of low-grade glioma.",
    "Administer 5mg of lorazepam intravenously for acute seizure management.",
    "Blood tests showed elevated ALT and AST levels, indicating possible liver inflammation.",
    "The subject reported chronic lower back pain, managed with physical therapy and NSAIDs.",
    "CT angiography confirmed the presence of a pulmonary embolism in the right lower lobe.",
    "The patient underwent coronary artery bypass graft surgery without complications.",
    "Routine vaccination included MMR, tetanus, and influenza immunizations.",
    "Histopathology indicated ductal carcinoma in situ (DCIS) in the breast biopsy sample.",
    "The child presented with a persistent cough and fever, diagnosed as streptococcal pharyngitis."
]'''

'docs = [\n    "The patient was diagnosed with type 2 diabetes mellitus and prescribed metformin 500mg twice daily.",\n    "MRI scan revealed a small lesion in the left temporal lobe suggestive of low-grade glioma.",\n    "Administer 5mg of lorazepam intravenously for acute seizure management.",\n    "Blood tests showed elevated ALT and AST levels, indicating possible liver inflammation.",\n    "The subject reported chronic lower back pain, managed with physical therapy and NSAIDs.",\n    "CT angiography confirmed the presence of a pulmonary embolism in the right lower lobe.",\n    "The patient underwent coronary artery bypass graft surgery without complications.",\n    "Routine vaccination included MMR, tetanus, and influenza immunizations.",\n    "Histopathology indicated ductal carcinoma in situ (DCIS) in the breast biopsy sample.",\n    "The child presented with a persistent cough and fever, diagnosed as streptococcal pharyngitis."\n]'

In [14]:
'''docs = [
    context,
    "The patient was diagnosed with type 2 diabetes mellitus and prescribed metformin 500mg twice daily.",
    "MRI scan revealed a small lesion in the left temporal lobe suggestive of low-grade glioma.",
]'''

'docs = [\n    context,\n    "The patient was diagnosed with type 2 diabetes mellitus and prescribed metformin 500mg twice daily.",\n    "MRI scan revealed a small lesion in the left temporal lobe suggestive of low-grade glioma.",\n]'

In [ ]:
rag.fit(new_docs[:100], dataset_name='movies')

creating graph


  0%|                                                   | 0/138 [00:00<?, ?it/s]Llama.generate: 2 prefix-match hit, remaining 1266 prompt tokens to eval


In [16]:
rag.graph.nodes

[Node(id=1, name='Stuart Rosenberg', description='American film and television director', alias=['Stuart Rosenberg'], additional_information=['August 11, 1927 – March 15, 2007']),
 Node(id=2, name='Paul Newman', description='Actor', alias=[], additional_information=[]),
 Node(id=3, name='Cool Hand Luke', description='Film', alias=[], additional_information=[]),
 Node(id=4, name='Voyage of the Damned', description='Film', alias=[], additional_information=[]),
 Node(id=5, name='The Amityville Horror', description='Film', alias=[], additional_information=[]),
 Node(id=6, name='The Pope of Greenwich Village', description='Film', alias=[], additional_information=[]),
 Node(id=7, name='Jean-Daniel Pollet', description='French film director', alias=[], additional_information=[]),
 Node(id=8, name='Volker Schlöndorff', description='German film director', alias=[], additional_information=[]),
 Node(id=9, name='Philippe Sollers', description='French writer', alias=[], additional_information=[]),

In [17]:
rag.graph.edges

[Edge(source=1, target=3, name='Directed', is_directed=True, description='Directed the film', notes=[]),
 Edge(source=1, target=4, name='Directed', is_directed=True, description='Directed the film', notes=[]),
 Edge(source=1, target=5, name='Directed', is_directed=True, description='Directed the film', notes=[]),
 Edge(source=1, target=6, name='Directed', is_directed=True, description='Directed the film', notes=[]),
 Edge(source=1, target=2, name='Worked with', is_directed=True, description='Worked with the actor', notes=[]),
 Edge(source=7, target=13, name='Directed', is_directed=True, description='Directed the film', notes=[]),
 Edge(source=7, target=14, name='Influenced', is_directed=True, description='Influenced the film', notes=[]),
 Edge(source=8, target=13, name='Assisted', is_directed=True, description='Assisted the director', notes=[]),
 Edge(source=9, target=13, name='Wrote', is_directed=True, description='Wrote the script', notes=[]),
 Edge(source=10, target=13, name='Produc

In [21]:
queries[0]

'How old is Jack?'

In [18]:
print('Question:', train[0]['question'])
resp = rag.query(train[0]['question'])
resp

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
entities ['Move', 'Méditerranée']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[np.int64(12), np.int64(14)]]
node_ids [tensor([4, 2, 6]), tensor([4, 3, 7])]
node_ids [4, 2, 6, 4, 3, 7]
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?

# Context:
KnowledgeGraph Overview
  There are 7 entities and 1 relationships in this graph.

Entities in the graph:
  • 'Paul Newman':
      The entity has 

llama_perf_context_print:        load time =    9934.85 ms
llama_perf_context_print: prompt eval time =   16438.24 ms /   566 tokens (   29.04 ms per token,    34.43 tokens per second)
llama_perf_context_print:        eval time =   44506.09 ms /    99 runs   (  449.56 ms per token,     2.22 tokens per second)
llama_perf_context_print:       total time =   61351.46 ms /   665 tokens


"Based on the information provided, it can be inferred that Jean-Daniel Pollet directed the film 'Méditerranée' [1]. However, there is no information about Jean-Daniel Pollet directing a film called 'Move' [2]. Therefore, it can be concluded that the director of the film 'Move' and the director of the film 'Méditerranée' are not the same. [1: The entity has the following description: French film director] [2"

In [24]:
print('Question:', queries[1])
resp = rag.query(queries[1])
resp

Question: What gender is Jill?Give me the character arc of Sam
entities ['Jill', 'Sam']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[np.int64(1), np.int64(2)]]
node_ids [tensor([9, 7]), tensor([4, 0, 5])]
node_ids [9, 7, 4, 0, 5]
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
What gender is Jill?Give me the character arc of Sam

# Context:
KnowledgeGraph Overview
  There are 6 entities and 7 relationships in this graph.

Entities in the graph:
  • 'Jill':
      The entity has the following description: Jill is an adult sibling of Jack.
      It can also be referred to as: Jill
      It has the following additional 

Llama.generate: 1005 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    9163.97 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    9254.43 ms /   100 runs   (   92.54 ms per token,    10.81 tokens per second)
llama_perf_context_print:       total time =    9668.53 ms /   101 tokens


'Jill is an adult sibling of Jack. As for the character arc of Sam, Sam is a friend of Jack and Jill, and they helped Sam find love. Sam then moved to Romania with their friend [1]. Sam found love in Romania, marking a significant change in their life [2].\n\nReferences:\n[1] Entity: Romania\n[2] Relationship: Sam moved to Romania with their friend\n\nNote: The answer is based on the information provided in the input data, and it is'

In [25]:
print('Question:', queries[2])
resp = rag.query(queries[2])
resp

Question: What location is the story set in
entities []


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[]]
node_ids []
node_ids []

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
What location is the story set in

# Context:
KnowledgeGraph Overview
  There are 0 entities and 0 relationships in this graph.

Entities in the graph:

Relationships between nodes:



# INSTRUCTIONS
Your goal is to provide a response to the user query using the relevant information in the input data:
- the "Entities" and "Relationships" tables contain high-level information. Use these tables to identify the most important entities and relationships to respond to the query.
- the "Sources" list contains raw text sources to help answer the query. It may contain noisy data, so pay attention when analyzing it.

Follow these steps:
1. Read and understand the user query.
2. Look at the "Entities" and "Relationships" tables to get a general sense of the data and understand which information is the most relevant to answer the query.
3. Carefu

Llama.generate: 28 prefix-match hit, remaining 333 prompt tokens to eval
llama_perf_context_print:        load time =    9163.97 ms
llama_perf_context_print: prompt eval time =    3022.72 ms /   333 tokens (    9.08 ms per token,   110.17 tokens per second)
llama_perf_context_print:        eval time =    4559.04 ms /    99 runs   (   46.05 ms per token,    21.72 tokens per second)
llama_perf_context_print:       total time =    7756.66 ms /   432 tokens


"Based on the given input data, no location is mentioned in the data. Therefore, it is not possible to determine the location of the story based on the provided information. [1]\n\nSource: [1] - There is no source ID provided in the input data. \n\nNote: I've followed the instructions and tried my best to provide a helpful response to the user query. Please let me know if I can improve anything. \n\nPlease let me know if the response is accurate and if there's"

In [30]:
print('Question:', queries[3])
resp = rag.query(queries[3])
resp

Question: What location is the story set in
entities []


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[]]
node_ids []
node_ids []

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
What location is the story set in

# Context:
KnowledgeGraph Overview
  There are 0 entities and 0 relationships in this graph.

Entities in the graph:

Relationships between nodes:



# INSTRUCTIONS
Your goal is to provide a response to the user query using the relevant information in the input data:
- the "Entities" and "Relationships" tables contain high-level information. Use these tables to identify the most important entities and relationships to respond to the query.
- the "Sources" list contains raw text sources to help answer the query. It may contain noisy data, so pay attention when analyzing it.

Follow these steps:
1. Read and understand the user query.
2. Look at the "Entities" and "Relationships" tables to get a general sense of the data and understand which information is the most relevant to answer the query.
3. Carefu

Llama.generate: 27 prefix-match hit, remaining 334 prompt tokens to eval
llama_perf_context_print:        load time =    9163.97 ms
llama_perf_context_print: prompt eval time =    3546.62 ms /   334 tokens (   10.62 ms per token,    94.17 tokens per second)
llama_perf_context_print:        eval time =    4649.51 ms /    99 runs   (   46.96 ms per token,    21.29 tokens per second)
llama_perf_context_print:       total time =    8388.49 ms /   433 tokens


"The story is set in [1]. [2] This information is based on the given input data. No other information is available for this query. If more information is provided, it could be possible to provide a more detailed answer.\n\nPlease provide more information about the story, as the current input data does not contain enough information to answer the query accurately. \n\n[1] <Source ID>\n[2] <Source ID> \n\nPlease provide the Source IDs and the actual source text. I'll"

In [29]:
print('Question:', queries[4])
resp = rag.query(queries[4])
resp

Question: Tell me how old Jack is, what the gender of Jill is, and then give me the character arcs of Sam and where the stoty is set
entities ['Jack', 'Jill', 'Sam']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[np.int64(0), np.int64(1), np.int64(2)]]
node_ids [tensor([4, 2]), tensor([9, 7]), tensor([4, 0, 5])]
node_ids [4, 2, 9, 7, 4, 0, 5]
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
Tell me how old Jack is, what the gender of Jill is, and then give me the character arcs of Sam and where the stoty is set

# Context:
KnowledgeGraph Overview
  There are 7 entities and 10 relationships in this graph.

Entities in the gra

Llama.generate: 57 prefix-match hit, remaining 1147 prompt tokens to eval
llama_perf_context_print:        load time =    9163.97 ms
llama_perf_context_print: prompt eval time =   10635.08 ms /  1147 tokens (    9.27 ms per token,   107.85 tokens per second)
llama_perf_context_print:        eval time =    4664.64 ms /    99 runs   (   47.12 ms per token,    21.22 tokens per second)
llama_perf_context_print:       total time =   15811.75 ms /  1246 tokens


"Based on the given data, we can answer the user query as follows:\nJack is 45 years old [1]. The gender of Jill is not explicitly mentioned in the given data, so we cannot determine it. As for Sam's character arc, we do not have enough information to provide a comprehensive analysis. The story is set in Romania [2], where Sam and their friend moved to. \n\nReferences:\n[1] - Entity description of Jack\n[2] - Relationship between Sam and Romania"